In [1]:
import re
import cloudscraper
import csv

In [2]:
url = 'https://www.muscleandstrength.com/exercises/'
muscle_groups = ["abductors.html", "abs", "adductors.html", "biceps", "calves", "chest", "forearms", "glutes", "hamstrings", "hip-flexors", "it-band", "lats", "lower-back", "middle-back", "neck.html", "obliques", "palmar-fascia", "plantar-fascia", "quads", "shoulders", "traps", "triceps"]
scraper = cloudscraper.create_scraper()

data = []
not_fetched = []
for muscle_group in muscle_groups:
    scraper_response = scraper.get(url + muscle_group + "/")
    pattern = "<div class=\"cell small-12 bp600-6\">\n<div class=\"node-image\">\n<a href=\"\/exercises\/(.*).html\">"
    matches = re.findall(pattern, scraper_response.text, re.MULTILINE)
    for match in matches:
        # print(muscle_group, match)
        try:
            scraper_response = scraper.get(url + "/" + match + ".html")
            regex = (r"<div class=\"node-stats-block\">\n"
                r"<h5>Exercise Profile<\/h5>\n"
                r"<ul>\n"
                r"<li><span class=\"row-label\">Target Muscle Group<\/span><div class=\"field field-name-field-main-muscle field-type-taxonomy-term-reference field-label-hidden\"><div class=\"field-items\"><div class=\"field-item even\"><a href=\"(.*)\">(.*)<\/a><\/div><\/div><\/div><\/li>\n"
                r"<li><span class=\"row-label\">Exercise Type<\/span>(.*)<\/li>\n"
                r"<li><span class=\"row-label\">Equipment Required<\/span>(.*)<\/li>\n"
                r"<li><span class=\"row-label\">Mechanics<\/span>(.*)<\/li>\n"
                r"<li><span class=\"row-label\">Force Type<\/span>(.*)<\/li>\n"
                r"<li><span class=\"row-label\">Experience Level<\/span>(.*)<\/li>\n"
                r"<li class=\"last\"><span class=\"row-label\">Secondary Muscles<\/span>\n"
                r"<div class=\"field-type-list-text\">\n"
                r"(.*) <\/div>\n"
                r"<\/li>\n"
                r"<\/ul>\n"
                r"<\/div>")
            test_str = scraper_response.text
            ex_data = re.findall(regex, test_str, re.MULTILINE)

            if "video/mp4" in test_str:
                link_regex = r"<source type=\"video/mp4\" src=\"//(.*)\">"
                video_type = "mp4"
            elif "www.youtube.com/embed" in test_str:
                link_regex = r"<iframe width=\"\d+\" height=\"\d+\" src=\"(.*?)\" "
                video_type = "youtube"
            else:
                link_regex = r"<div style=\"padding:\d+\.\d+% 0 0 0;position:relative;\"><iframe src=\"(.*?)\" "
                video_type = "vimeo"

            link_matches = re.findall(link_regex, test_str)
            # print(ex_data, link_matches)
            data.append({"title": match.replace("-", " "),"target_muscle_group": ex_data[0][1], "exercise_type": ex_data[0][2], "equipment": ex_data[0][3], "mechanics": ex_data[0][4], "force_type": ex_data[0][5], "experience_level": ex_data[0][6], "secondary_muscles": ex_data[0][7], "video_url": link_matches[0], "video_type": video_type})
        except:
            print(muscle_group, match, ex_data, link_matches)
            not_fetched.append(match)

field_names = data[0].keys()
with open('fit_data.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows(data)

In [35]:
scraper = cloudscraper.create_scraper()
count = 0
exes = []
while True:
    count += 1
    url = "https://www.bodybuilding.com/exercises/finder/{count}?undefined=".format(count=count)
    try:
        scraper_response = scraper.get(url)
        test_str = scraper_response.text
        link_regex = r"<a href=\"/exercises/(.*)\" itemprop=\"name\">"
        matches = re.findall(link_regex, test_str)
        if len(matches) == 0:
            break
        exes += matches
    except:
        break

print(len(exes))


3149


In [87]:
title_regex = r"\s+<h1 class=\"ExHeading ExHeading--h2 ExDetail-h2\" itemprop=\"name\">\n\s+(.*)\n\s+</h1>"

exercise_type_regex = (r"\s+<div class=\"grid-3 grid-12-s grid-8-m\">\n"
	r"\s+<ul class=\"bb-list--plain\">\n"
	r"\s+<li>\n"
	r"\s+Type:\n"
	r"\s+<a href=\".+\" itemprop=\"exerciseType\">\n"
	r"\s+(.*)\n"
	r"\s+</a>\n"
	r"\s+</li>\n")

target_muscle_group_regex = (r"\s+<li>\n"
	r"\s+Main Muscle Worked:\n"
	r"\s+<a href=\".+\">\n"
	r"\s+(.*)\n"
	r"\s+</a>\n"
	r"\s+</li>\n")

equipment_regex = (r"\s+<li>\n"
	r"\s+Equipment:\n"
	r"\s+<a href=\".+\">\n"
	r"\s+(.*)\n"
	r"\s+</a>\n"
	r"\s+</li>\n")

experience_level_regex = (r"\s+<li>\n"
	r"\s+Level:\n"
	r"\s+(.*)\n"
	r"\s+</li>\n")

exercise_rating_regex = (r"\s+</div>\n"
	r"\s+<div class=\"grid-3 grid-12-s grid-4-m\">\n"
	r"\s+<div class=\"ExRating\">\n"
	r"\s+<div class=\"ExRating-badge\">\n"
	r"\s+(.*)\n"
	r"\s+</div>\n")

rating_desc_regex = (
	r"\s+<div class=\"ExRating-description .+\">\n"
	r"\s+(.*)\n"
	r"\s+</div>\n"
	r"\s+</div>\n"
	r"\s+</div>\n"
	r"\s+</section>\n"
	r"\s+<section class=\"ExDetail-section ExDetail-photos paywall__xdb-details\">")

alternative_regex = (r"\s+<h3 class=\"ExHeading ExResult-resultsHeading\">\n"
	r"\s+<a href=\".+\" itemprop=\"name\">\n"
	r"\s+(.*)\n"
	r"\s+</a>\n"
	r"\s+</h3>")

data = []
not_fetched = []

for exercise in exes:
	try:
		url = "https://www.bodybuilding.com/exercises/" + exercise
		scraper_response = scraper.get(url)
		test_str = scraper_response.text

		title_match = re.findall(title_regex, test_str)
		exercise_type_match = re.findall(exercise_type_regex, test_str)
		target_muscle_group_match = re.findall(target_muscle_group_regex, test_str)
		equipment_match = re.findall(equipment_regex, test_str)
		experience_level_match = re.findall(experience_level_regex, test_str)
		exercise_rating_match = re.findall(exercise_rating_regex, test_str)
		rating_desc_match = re.findall(rating_desc_regex, test_str)
		alternative_match = re.findall(alternative_regex, test_str)

		if len(exercise_type_match): exercise_type = exercise_type_match[0]
		else: rating_desc = "N/A"

		if len(target_muscle_group_match): target_muscle_group = target_muscle_group_match[0]
		else: target_muscle_group = "N/A"

		if len(equipment_match): equipment = equipment_match[0]
		else: equipment = "N/A"

		if len(experience_level_match): experience_level = experience_level_match[0]
		else: experience_level = "N/A"

		if len(exercise_rating_match): exercise_rating = exercise_rating_match[0]
		else: exercise_rating = "N/A"

		if len(rating_desc_match): rating_desc = rating_desc_match[0]
		else: rating_desc = "N/A"

		if len(alternative_match): alternative_exercises = ",".join(alternative_match)
		else: alternative_exercises = "N/A"


		data.append({"title": title_match[0],"exercise_type": exercise_type, "target_muscle_group": target_muscle_group, "equipment": equipment, "experience_level": experience_level, "exercise_rating": exercise_rating, "exercise_rating_description": rating_desc, "alternative_exercises": alternative_exercises})
	except:
		not_fetched.append(exercise)
		print(exercise)

print(len(data))

pistol-squat
smith-single-leg-split-squat
fyr-goblet-squat-plus-plank-up
fyr-medicine-ball-jumping-sumo-squat
fyr-hollow-body-hold
fyr-double-kettlebell-push-press
exercise-ball-oblique-crunch
poston-press
poston-press
exercise-ball-oblique-crunch
seated-dumbbell-overhead-triceps-press
fyr-dumbbell-overhead-hold
hm-up-dog-pose
taylor-single-arm-snatch-to-get-up
holman-alternating-knee-pull-in
holman-overhead-press-to-chest-press
total-fitness-hollow-rock
single-arm-band-overhead-triceps-extension
band-stiff-legged-deadlift
3130


In [88]:
field_names = data[0].keys()
with open('body_building_data.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows(data)

In [91]:
url = "https://www.bodybuilding.com/exercises/" + exes[5]

scraper_response = scraper.get(url)
test_str = scraper_response.text

f = open("temp.html", "w")
f.write(scraper_response.content.decode('UTF-8'))
f.close()

title_regex = r"\s+<h1 class=\"ExHeading ExHeading--h2 ExDetail-h2\" itemprop=\"name\">\n\s+(.*)\n\s+</h1>"
info_regex = (r"\s+<div class=\"grid-3 grid-12-s grid-8-m\">\n"
	r"\s+<ul class=\"bb-list--plain\">\n"
	r"\s+<li>\n"
	r"\s+Type:\n"
	r"\s+<a href=\".+\" itemprop=\"exerciseType\">\n"
	r"\s+(.*)\n"
	r"\s+</a>\n"
	r"\s+</li>\n"
	r"\s+<li>\n"
	r"\s+Main Muscle Worked:\n"
	r"\s+<a href=\".+\">\n"
	r"\s+(.*)\n"
	r"\s+</a>\n"
	r"\s+</li>\n"
	r"\s+<li>\n"
	r"\s+Equipment:\n"
	r"\s+<a href=\".+\">\n"
	r"\s+(.*)\n"
	r"\s+</a>\n"
	r"\s+</li>\n"
	r"\s+<li>\n"
	r"\s+Level:\n"
	r"\s+(.*)\n"
	r"\s+</li>\n"
	r"\s+</ul>\n"
	r"\s+</div>\n"
	r"\s+<div class=\"grid-3 grid-12-s grid-4-m\">\n"
	r"\s+<div class=\"ExRating\">\n"
	r"\s+<div class=\"ExRating-badge\">\n"
	r"\s+(.*)\n"
	r"\s+</div>\n")

rating_desc_regex = (
	r"\s+<div class=\"ExRating-description .+\">\n"
	r"\s+(.*)\n"
	r"\s+</div>\n"
	r"\s+</div>\n"
	r"\s+</div>\n"
	r"\s+</section>\n"
	r"\s+<section class=\"ExDetail-section ExDetail-photos paywall__xdb-details\">")

alternative_regex = (r"\s+<h3 class=\"ExHeading ExResult-resultsHeading\">\n"
	r"\s+<a href=\".+\" itemprop=\"name\">\n"
	r"\s+(.*)\n"
	r"\s+</a>\n"
	r"\s+</h3>")

title_matches = re.findall(title_regex, test_str)
info_matches = re.findall(info_regex, test_str)
rating_desc_match = re.findall(rating_desc_regex, test_str)
alternative_match = re.findall(alternative_regex, test_str)

print(title_matches, info_matches, rating_desc_match, alternative_match)

['Palms-down wrist curl over bench'] [('Strength', 'Forearms', 'Barbell', 'Intermediate', '9.5')] ['Average'] ['Palms-Down Dumbbell Wrist Curl Over A Bench', 'Seated palms-down wrist curl', 'Seated One-Arm Dumbbell Palms-Down Wrist Curl']


In [3]:
import requests
from bs4 import BeautifulSoup

# URL of the page to scrape
url = 'https://exrx.net/Lists/Directory'

# Send an HTTP request to the URL and get the HTML content
response = requests.get(url)
html_content = response.content

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table with the exercise data
table = soup.find('table', {'class': 'exercise-table'})

# Get a list of all rows in the table
rows = table.find_all('tr')

# Loop through each row and extract the data
for row in rows:
    # Get the cells in the row
    cells = row.find_all('td')
    
    # Extract the data from each cell
    exercise_name = cells[0].get_text()
    primary_muscles = cells[1].get_text()
    secondary_muscles = cells[2].get_text()
    equipment = cells[3].get_text()
    intensity = cells[4].get_text()
    
    # Do something with the data, such as printing it
    print(exercise_name, primary_muscles, secondary_muscles, equipment, intensity)


AttributeError: 'NoneType' object has no attribute 'find_all'

In [4]:
soup

<html><body>Error communicating with origin</body></html>